# Importar Bibliotecas Necesarias
Importar las bibliotecas necesarias, incluyendo json, re, y collections.

In [ ]:
# Importar las bibliotecas necesarias
import json
import re
from collections import defaultdict
from nltk.corpus import stopwords
import nltk
# Descargar las stopwords en español
#nltk.download('stopwords')

# Leer y Procesar el Corpus de Texto
Leer el archivo pcovid2020.json y procesar cada línea para extraer el texto de los tweets.

In [ ]:

stop_words = set(stopwords.words('spanish'))

# Leer y Procesar el Corpus de Texto

# Definir la función para leer y procesar el archivo JSON
def leer_corpus(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        corpus = [json.loads(line)['text'] for line in f]
    return corpus

# Procesar el texto de los tweets
def procesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Remover puntuaciones y acentos
    texto = re.sub(r'[^\w\s]', '', texto)
    # Tokenizar el texto
    palabras = texto.split()
    # Eliminar stopwords
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    return palabras

# Leer el archivo y procesar cada línea
archivo = 'pcovid2020.json'
corpus = leer_corpus(archivo)
corpus_procesado = [procesar_texto(texto) for texto in corpus]

# Mostrar un ejemplo del corpus procesado
print(corpus_procesado[:1])

[['gran', 'reto', 'terapéutico', 'covid19', 'domar', 'sistema', 'inmunitario', 'frente', 'coronavirus', 'httpstcoabcvduerjx', 'vía', 'eldiarioes']]


# Construir el Índice Invertido
Construir un índice invertido simple con listas de posteo representadas por listas ordenadas de enteros, asociadas a palabras válidas.

In [15]:
# Construir el Índice Invertido

# Crear el índice invertido
indice_invertido = defaultdict(list)

# Asignar un número a cada palabra usando la función hash
for doc_id, texto in enumerate(corpus_procesado):
    for palabra in texto:
        palabra_hash = hash(palabra)
        if doc_id not in indice_invertido[palabra_hash]:
            indice_invertido[palabra_hash].append(doc_id)

# Ordenar las listas de posteo
for palabra_hash in indice_invertido:
    indice_invertido[palabra_hash].sort()

# Mostrar un ejemplo del índice invertido
print(dict(list(indice_invertido.items())[:5]))

{1852313151933298543: [0, 43, 117, 234, 303, 330, 712, 868, 1172, 1295, 1446, 1688, 1793, 1878, 1983, 2216, 2309, 2400, 2485, 2512, 2650, 2658, 2777, 2795, 3117, 3177, 3267, 3279, 3566, 3654, 3661, 3987, 4243, 4599, 4792, 4877, 4922, 4937, 4950, 4990, 5139, 5281, 5434, 5463, 5491, 5649, 5994, 6049, 6155, 6183, 6239, 6297, 6390, 6548, 6812, 6846, 7071, 7287, 7298, 7391, 7566, 7623, 7646, 7780, 7797, 7816, 7839, 7989, 8200, 8205, 8206, 8251, 8397, 8467, 8507, 8511, 8635, 8692, 8854, 8856, 9030, 9087, 9166, 9421, 9671, 9693, 9937, 9996], 855326066358006271: [0, 1231, 1744, 1942, 3117, 3349, 3523, 4679, 5143, 9777], 223701025582650719: [0, 3000], -8018376882435497838: [0, 3, 10, 12, 16, 28, 29, 31, 44, 45, 47, 48, 50, 52, 55, 57, 58, 59, 66, 68, 71, 73, 75, 84, 86, 88, 91, 96, 99, 100, 101, 103, 106, 107, 108, 112, 115, 118, 120, 122, 124, 126, 127, 135, 140, 141, 143, 144, 147, 155, 156, 160, 163, 166, 168, 171, 177, 180, 182, 186, 189, 195, 198, 199, 200, 203, 215, 217, 218, 219, 228, 23

# Definir Funciones para Consultas Conjuntivas
Definir funciones para resolver consultas conjuntivas utilizando algoritmos de intersección, como búsqueda binaria y búsqueda galloping.

In [16]:
# Definir Funciones para Consultas Conjuntivas

# Función para realizar búsqueda binaria
def busqueda_binaria(lista, elemento):
    izquierda, derecha = 0, len(lista) - 1
    while izquierda <= derecha:
        medio = (izquierda + derecha) // 2
        if lista[medio] == elemento:
            return medio
        elif lista[medio] < elemento:
            izquierda = medio + 1
        else:
            derecha = medio - 1
    return -1

# Función para realizar búsqueda galloping
def busqueda_galloping(lista, elemento):
    if not lista:
        return -1
    if lista[0] == elemento:
        return 0
    i = 1
    while i < len(lista) and lista[i] < elemento:
        i *= 2
    izquierda, derecha = i // 2, min(i, len(lista))
    return busqueda_binaria(lista[izquierda:derecha], elemento)

# Función para intersección de listas usando búsqueda binaria
def interseccion_binaria(lista1, lista2):
    resultado = []
    for elemento in lista1:
        if busqueda_binaria(lista2, elemento) != -1:
            resultado.append(elemento)
    return resultado

# Función para intersección de listas usando búsqueda galloping
def interseccion_galloping(lista1, lista2):
    resultado = []
    for elemento in lista1:
        if busqueda_galloping(lista2, elemento) != -1:
            resultado.append(elemento)
    return resultado

# Función para resolver consultas conjuntivas
def resolver_consulta(consulta, metodo='binaria'):
    palabras = procesar_texto(consulta)
    listas_posteo = [indice_invertido[hash(palabra)] for palabra in palabras if hash(palabra) in indice_invertido]
    
    if not listas_posteo:
        return []
    
    listas_posteo.sort(key=len)
    resultado = listas_posteo[0]
    
    for lista in listas_posteo[1:]:
        if metodo == 'binaria':
            resultado = interseccion_binaria(resultado, lista)
        elif metodo == 'galloping':
            resultado = interseccion_galloping(resultado, lista)
    
    return resultado

# Ejemplo de uso de la función resolver_consulta
consulta = "vacuna covid"
resultados = resolver_consulta(consulta, metodo='binaria')
print(resultados)

[399, 438, 1039, 1790, 2194, 2216, 2753, 3283, 3771, 5843, 6356, 6636, 6771, 7192, 7424, 7666, 7741, 8467, 8723]


# Construir el Índice Invertido en una Celda
Crear una celda en el notebook donde se construya el índice invertido utilizando las funciones definidas.

In [17]:
# Construir el Índice Invertido en una Celda

# Crear el índice invertido
indice_invertido = defaultdict(list)

# Asignar un número a cada palabra usando la función hash
for doc_id, texto in enumerate(corpus_procesado):
    for palabra in texto:
        palabra_hash = hash(palabra)
        if doc_id not in indice_invertido[palabra_hash]:
            indice_invertido[palabra_hash].append(doc_id)

# Ordenar las listas de posteo
for palabra_hash in indice_invertido:
    indice_invertido[palabra_hash].sort()

# Mostrar un ejemplo del índice invertido
print(dict(list(indice_invertido.items())[:5]))

{1852313151933298543: [0, 43, 117, 234, 303, 330, 712, 868, 1172, 1295, 1446, 1688, 1793, 1878, 1983, 2216, 2309, 2400, 2485, 2512, 2650, 2658, 2777, 2795, 3117, 3177, 3267, 3279, 3566, 3654, 3661, 3987, 4243, 4599, 4792, 4877, 4922, 4937, 4950, 4990, 5139, 5281, 5434, 5463, 5491, 5649, 5994, 6049, 6155, 6183, 6239, 6297, 6390, 6548, 6812, 6846, 7071, 7287, 7298, 7391, 7566, 7623, 7646, 7780, 7797, 7816, 7839, 7989, 8200, 8205, 8206, 8251, 8397, 8467, 8507, 8511, 8635, 8692, 8854, 8856, 9030, 9087, 9166, 9421, 9671, 9693, 9937, 9996], 855326066358006271: [0, 1231, 1744, 1942, 3117, 3349, 3523, 4679, 5143, 9777], 223701025582650719: [0, 3000], -8018376882435497838: [0, 3, 10, 12, 16, 28, 29, 31, 44, 45, 47, 48, 50, 52, 55, 57, 58, 59, 66, 68, 71, 73, 75, 84, 86, 88, 91, 96, 99, 100, 101, 103, 106, 107, 108, 112, 115, 118, 120, 122, 124, 126, 127, 135, 140, 141, 143, 144, 147, 155, 156, 160, 163, 166, 168, 171, 177, 180, 182, 186, 189, 195, 198, 199, 200, 203, 215, 217, 218, 219, 228, 23

# Especificar Consultas
Crear una celda para especificar las consultas conjuntivas que se desean resolver.

In [21]:
# Especificar Consultas

# Crear una celda para especificar las consultas conjuntivas que se desean resolver
consultas = [
    "vacuna covid",
    "mascarilla prevención",
    "síntomas fiebre",
    "distanciamiento social",
    "tratamiento hospital"
]

# Resolver cada consulta y mostrar los resultados
for consulta in consultas:
    resultados = resolver_consulta(consulta, metodo='binaria')
    print(f"Consulta: '{consulta}' -> Resultados: {resultados}")

Consulta: 'vacuna covid' -> Resultados: [399, 438, 1039, 1790, 2194, 2216, 2753, 3283, 3771, 5843, 6356, 6636, 6771, 7192, 7424, 7666, 7741, 8467, 8723]
Consulta: 'mascarilla prevención' -> Resultados: []
Consulta: 'síntomas fiebre' -> Resultados: [3639, 5503]
Consulta: 'distanciamiento social' -> Resultados: [167, 674, 1216, 1574, 1605, 2501, 3021, 3607, 3912, 4118, 4244, 4816, 5782, 6371, 6789, 7087, 7668, 8336, 9235, 9538]
Consulta: 'tratamiento hospital' -> Resultados: [7307]


# Mostrar Resultados de las Consultas
Crear una celda para mostrar los resultados de las consultas conjuntivas resueltas.

In [23]:
# Mostrar Resultados de las Consultas

# Crear una celda para mostrar los resultados de las consultas conjuntivas resueltas
print("Binaria")
for consulta in consultas:
    resultados = resolver_consulta(consulta, metodo='binaria')
    print(f"Consulta: '{consulta}' -> Resultados: {resultados}")
    
print("\nGalloping")
for consulta in consultas:
    resultados = resolver_consulta(consulta, metodo='galloping')
    print(f"Consulta: '{consulta}' -> Resultados: {resultados}")

Binaria
Consulta: 'vacuna covid' -> Resultados: [399, 438, 1039, 1790, 2194, 2216, 2753, 3283, 3771, 5843, 6356, 6636, 6771, 7192, 7424, 7666, 7741, 8467, 8723]
Consulta: 'mascarilla prevención' -> Resultados: []
Consulta: 'síntomas fiebre' -> Resultados: [3639, 5503]
Consulta: 'distanciamiento social' -> Resultados: [167, 674, 1216, 1574, 1605, 2501, 3021, 3607, 3912, 4118, 4244, 4816, 5782, 6371, 6789, 7087, 7668, 8336, 9235, 9538]
Consulta: 'tratamiento hospital' -> Resultados: [7307]

Galloping
Consulta: 'vacuna covid' -> Resultados: [438, 1039, 1790, 2194, 2216, 2753, 3283, 3771, 5843, 6356, 6636, 6771, 7192, 7424, 7666, 7741, 8467]
Consulta: 'mascarilla prevención' -> Resultados: []
Consulta: 'síntomas fiebre' -> Resultados: [3639, 5503]
Consulta: 'distanciamiento social' -> Resultados: [674, 1216, 1574, 1605, 2501, 3021, 3607, 3912, 4118, 4244, 4816, 5782, 6371, 6789, 7087, 7668, 8336, 9235, 9538]
Consulta: 'tratamiento hospital' -> Resultados: [7307]
